In [1]:
import os
os.chdir('..')

In [2]:
import pickle as pkl
import pandas as pd
import numpy as np
import dalex

In [3]:
with open('models/xgboost.pkl', 'rb') as f:
    model = pkl.load(f)
with open('models/relevant_cols.pkl', 'rb') as f:
    relevant_cols = pkl.load(f)

/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
df_test = pd.read_csv('data/test_processed.csv')
X_test, y_test = df_test.drop(columns='FULLVAL')[relevant_cols], df_test['FULLVAL']

# 1. Metrics

In [5]:
from sklearn.metrics import(
    mean_absolute_error,
    r2_score
)

In [6]:
mean_absolute_error(y_test, model.predict(X_test))

/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


201118.19755227226

In [7]:
r2_score(y_test, model.predict(X_test))

/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.9712933551188573

# 2. Decile Analysis

In [8]:
pred = model.predict(X_test)
df_summary = pd.DataFrame({
    'deciles': pd.qcut(pred, q=10),
    'actual': y_test,
    'predicted': pred
})
df_summary = df_summary.assign(
    relative_error=np.abs(df_summary.actual - df_summary.predicted)/df_summary.actual
)

/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [9]:
df_summary.groupby('deciles', as_index=False).mean()

,deciles,actual,predicted,relative_error
0,"(19138.043999999998, 181996.675]",6.502813e+04,1.012256e+05,NaN
1,"(181996.675, 273549.781]",2.218077e+05,2.360055e+05,0.203514
2,"(273549.781, 396399.828]",3.166781e+05,3.152221e+05,0.255660
3,"(396399.828, 461669.988]",4.112371e+05,4.319131e+05,0.127607
4,"(461669.988, 510274.938]",4.632674e+05,4.844921e+05,0.131419
5,"(510274.938, 586370.938]",5.500624e+05,5.452001e+05,0.120442
6,"(586370.938, 642609.669]",6.017908e+05,6.129769e+05,0.107255
7,"(642609.669, 951005.763]",7.361155e+05,7.640543e+05,0.151569
8,"(951005.763, 1580098.388]",1.208394e+06,1.242411e+06,0.257472
9,"(1580098.388, 285254112.0]",1.049581e+07,9.779332e+06,0.171868


# 3. Model explanation

In [10]:
explainer = dalex.Explainer(model, X_test, y_test)

Preparation of a new explainer is initiated

  -> data              : 1792 rows 41 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 1792 values
  -> model_class       : xgboost.sklearn.XGBRFRegressor (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_default at 0x7f4c8b63be50> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 1.91e+04, mean = 1.46e+06, max = 2.85e+08
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.34e+07, mean = 5.61e+04, max = 8.37e+07
  -> model_info        : package xgboost

A new explainer has been created!


/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [11]:
bd = dalex.predict_explanations.BreakDown()

In [12]:
bd.fit(explainer, X_test.loc[69])

In [14]:
bd.plot(max_vars=50)

In [26]:
sh = dalex.predict_explanations.Shap()

In [33]:
sh.fit(explainer, X_test.loc[70])

/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

/home/antoni/miniconda3/envs/best/lib/python3.9/

In [34]:
sh.plot(max_vars=50)

In [36]:
df_test.loc[69]

LTFRONT             0.205251
LTDEPTH             0.004115
STORIES            -2.425673
FULLVAL        104000.000000
AVLAND             -0.893186
                   ...      
TAXCLASS_2B         0.000000
TAXCLASS_2C         0.000000
TAXCLASS_3          0.000000
TAXCLASS_4          0.000000
loc_cluster         1.000000
Name: 69, Length: 124, dtype: float64

In [44]:
cp = dalex.predict_explanations.CeterisParibus()
cp.fit(explainer, X_test.loc[69], np.array(y_test[69]))
cp.plot(variables=['AVLAND', 'AVTOT'])

Calculating ceteris paribus:   0%|          | 0/41 [00:00<?, ?it/s]/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

/home/antoni/miniconda3/envs/best/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropria